In [ ]:
!pip install yt-dlp

In [ ]:
import os
import subprocess
from IPython.display import Audio, display
import librosa

def download_and_play_youtube_audio(url, output_path="output.mp3"):
    try:
        # Use yt-dlp to download the audio
        subprocess.run([
            'yt-dlp',
            '-x', '--audio-format', 'mp3',
            '-o', 'temp_audio.%(ext)s',
            url
        ], check=True)

        temp_file = "temp_audio.mp3"

        # Convert to the desired output path if needed
        if output_path != temp_file:
            os.rename(temp_file, output_path)

        # Load the audio file using librosa
        audio_data, sr = librosa.load(output_path, sr=None)

        # Display audio player in Kaggle notebook
        print(f"Playing audio from: {output_path}")
        display(Audio(audio_data, rate=sr))

        return True

    except Exception as e:
        print(f"Error: {str(e)}")

url = input("SENIORS PLZ ENTER THE YOUTUBE URL:")
download_and_play_youtube_audio(url, "output.mp3")

In [ ]:
!pip install openai-whisper


In [ ]:
import whisper

def transcribe_audio_with_whisper(audio_path):
    try:
        # Load the Whisper model
        model = whisper.load_model("base")

        # Transcribe the audio directly using Whisper 
        result = model.transcribe(audio_path)

        # Return the transcribed text instead of printing directly
        return result['text']

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Call the function with the audio path after downloading
audio_path = "output.mp3"  # Path to your downloaded audio file
transcription = transcribe_audio_with_whisper(audio_path)

# Print the transcription explicitly here
if transcription:
    print(f"Transcription: {transcription}")


The transcription needs to be split into segments with timestamps so that you can overlay them on the video. Each subtitle block in an SRT file includes:

A sequence number.
A timestamp (start and end times).
The transcribed text.
SRT:SubRip Subtitle format

Overlay Subtitles Using FFmpeg: After generating the SRT file,  use FFmpeg to merge the subtitles with your video.

In [ ]:
import os

def create_srt_file(transcription_result, srt_path="subtitles.srt"):
    try:
        segments = transcription_result['segments']  # Whisper provides timestamps here
        with open(srt_path, "w", encoding="utf-8") as srt_file:
            for idx, segment in enumerate(segments):
                start_time = segment['start']
                end_time = segment['end']
                text = segment['text']

                # Format time as hh:mm:ss,ms
                def format_time(seconds):
                    hours = int(seconds // 3600)
                    minutes = int((seconds % 3600) // 60)
                    seconds = int(seconds % 60)
                    milliseconds = int((seconds - int(seconds)) * 1000)
                    return f"{hours:02}:{minutes:02}:{seconds:02},{milliseconds:03}"

                srt_file.write(f"{idx + 1}\n")
                srt_file.write(f"{format_time(start_time)} --> {format_time(end_time)}\n")
                srt_file.write(f"{text.strip()}\n\n")

        print(f"Subtitle file saved to: {srt_path}")
        return srt_path

    except Exception as e:
        print(f"Error creating SRT file: {str(e)}")
        return None

# Example usage
audio_path = "output.mp3"
transcription_result = whisper.load_model("base").transcribe(audio_path)
if transcription_result:
    srt_path = create_srt_file(transcription_result)


In [ ]:
def read_srt_file(srt_path):
    try:
        with open(srt_path, "r", encoding="utf-8") as file:
            content = file.read()
            print(content)
    except Exception as e:
        print(f"Error reading SRT file: {str(e)}")

# Example usage
srt_path = "subtitles.srt"
read_srt_file(srt_path)


In [ ]:
import subprocess

def download_video(url, output_path="input_video.mp4"):
    try:
        subprocess.run([
            'yt-dlp',
            '-f', 'bestvideo+bestaudio',
            '--merge-output-format', 'mp4',
            '-o', output_path,
            url
        ], check=True)
        print(f"Video downloaded successfully: {output_path}")
    except Exception as e:
        print(f"Error downloading video: {str(e)}")


In [ ]:
download_video(url)


In [ ]:
import subprocess

def overlay_subtitles(video_path, srt_path, output_path="output_video.mp4"):
    try:
        # Use FFmpeg to overlay subtitles
        subprocess.run([
            'ffmpeg',
            '-i', video_path,
            '-vf', f"subtitles={srt_path}",
            '-c:a', 'copy',
            output_path
        ], check=True)
        print(f"Subtitled video saved as: {output_path}")
        return output_path
    except Exception as e:
        print(f"Error adding subtitles: {str(e)}")
        return None

# Example usage
video_path = "input_video.mp4"  # Your downloaded video
srt_path = "subtitles.srt"      # Your generated subtitle file
overlay_subtitles(video_path, srt_path)



In [ ]:
!ls   #The output lists the files and directories present in the current working directory of your notebook.


In [ ]:
from IPython.display import Video

# Path to your video file
video_path = "output_video.mp4"

# Display the video
Video(video_path, embed=True, width=640, height=360)
